In [248]:
# para buscar archivos
from glob import glob
# super excel para analizar
import pandas as pd
# para leer los datos .json
import json
# para hacer busquedas en strings
import re
# para operaciones numericas
import numpy as np
# para que quede mas lindo
import warnings
warnings.filterwarnings('ignore')

In [249]:
# Para bajar los datos (Mati)
# aws s3 sync s3://unket/gyani ./data

In [250]:
def data2df(f):
    """recibe path al archivo y devuelve DataFrame"""
    with open(f, 'rb') as inp:
        j = inp.read()
        #j = re.sub(r"\\+", r"\\", j)
        data = json.loads(j)
        d = json.loads(data)
        df = pd.read_json(d)
    return df #pd.read_json(j)

In [251]:
files = glob('./datos/*.json')

In [252]:
def wrangle(row):
    etapa = etapas[int(re.search('-(\d+)', row['internal_node_id']).group(1))]
    try:
        responses = json.loads(row['responses'])
        return {f"{etapa}-"+ re.search('(\d+)', k).group(0) if k != 'undefined' else etapa: v for k,v in responses.items()}
    except:
        return {etapa:row['responses']}

In [253]:
def get_sujeto(df):
    return pd.DataFrame.from_dict({k:v for l in df.apply(wrangle, axis=1) for k,v in l.items()}, orient='index').T

In [254]:
etapas=['welcome', 'consent', 'demography1', 'demography2', 'if_otroo',
        'theoretical_orientation_ticks', 'if_tcc', 'if_psan', 'if_hum', 'if_ecl',
        'if_etc', 'workplace_ticks', 'if_otro', 'influence', 'sources',
        'research_attitudes_1', 'research_attitudes_2', 'comments']

In [255]:
f = files[10]

In [260]:
df = pd.DataFrame()
for f in files:
    df_ = data2df(f)
    df_['etapa'] = df_.apply(lambda x: etapas[int(re.search('-(\d+)', x['internal_node_id']).group(1))], axis=1)
    df=df.append(get_sujeto(df_))
    break

In [261]:
df_

,rt,stimulus,button_pressed,trial_type,trial_index,time_elapsed,internal_node_id,responses,etapa
0,2016.000,<font size=5.5><p>¡Hola!</p><p><p>Esta encuest...,0.0,html-button-response,0,2032,0.0-0.0,NaN,welcome
1,3326.000,<font size=5.5><p>Continuando con el experimen...,0.0,html-button-response,1,5361,0.0-1.0,NaN,consent
2,3681.535,NaN,NaN,survey-text,2,9060,0.0-2.0,"{""undefined"":"" ""}",demography1
3,1673.000,NaN,NaN,survey-multi-choice,3,10745,0.0-3.0,"{""Q0"":""Femenino"",""Q1"":""Licenciatura de grado""}",demography2
4,2909.000,NaN,NaN,survey-multi-choice,4,13667,0.0-5.0,"{""Q0"":""Ecléctica (más de una/de todo un poco/d...",theoretical_orientation_ticks
5,1574.410,NaN,NaN,survey-text,5,15248,0.0-9.0-0.0,"{""undefined"":"" ""}",if_ecl
6,3702.000,NaN,NaN,survey-multi-select,6,18956,0.0-11.0,"{""Q0"":[""Obra Social"",""Otro""]}",workplace_ticks
7,1475.560,NaN,NaN,survey-text,7,20437,0.0-12.0-0.0,"{""undefined"":"" ""}",if_otro
8,5698.000,NaN,NaN,survey-multi-choice,8,26159,0.0-13.0,"{""Q0"":""No influyó para nada"",""Q1"":""No influyó ...",influence
9,3291.000,NaN,NaN,survey-multi-choice,9,29481,0.0-14.0,"{""Q0"":""Nunca"",""Q1"":""Nunca"",""Q2"":""Nunca"",""Q3"":""...",sources


In [25]:
def concat_rows(df):
    rows = df.iloc[0,:]
    for i in range(len(df)-1):
        rows = pd.concat([rows, df.iloc[i+1,:]])
    return rows

In [49]:
concatenated = concat_rows(response)

In [48]:
pages = ['ins','demograficos', 'demografico 2', 'orientacion','sector', 'eleccion_orientacion']
demograficos = ['edad', 'exp_clinica', 'horasXsemana']
demograficos2 = ['genero', 'Nivel_educativo']
orientacion = ['orientacion']
sector = ['sector']
eleccion_orientacion = ['evidencia','pers_experiencia', 'entrenamiento',
                        'pref_consult', 'intuicion', 'docentes', 'pers_terapia']
mejorar = ['exp_pers_consult', 'inv_control', 'supervision', 'inv_casos',
           'pares','libros', 'obs_casos', 'medidas_resultados', 'guias_clinicas' ]
opinion1 = ['dispuesto_cambio', 'cons_mas_compejos', 'estar_al_dia', 
 'cons_otros_trat', 'formacion_inv', 'ter_no_manual', 'superv_evidencia',
 'diag_simples', 'tbe_eficientes', 'trat_no_probados', 'trat_individual']
opinion2 = ['incorporar_ciencia', 'tiempo_tbe', 'intentar_dif', 
            'capacit_costosa', 'interes_aorender', 'no_se_tbe', 'uso_tbe', 'entr_no_tbe',
           'alianza', 'ter_iguales', 'empleador_sin_fondos', 'exp_mejor_evi']
coments = ['preg_imprecisa', 'otras_pregs', 'otro']

cols = demograficos + (12-len(demograficos))*['NaN'] + demograficos2 + (12-len(demograficos2))*['NaN'] + orientacion+ (12-len(orientacion))*['NaN']+ ['otro_orient']+ 11*['NaN']+ sector+(12-len(sector))*['NaN'] + ['otro_sector']+ 11*['NaN']+eleccion_orientacion + (12-len(eleccion_orientacion))*['NaN']+ mejorar +(12-len(mejorar))*['NaN']+ opinion1 + (12-len(opinion1))*['NaN']+opinion2 +(12-len(opinion2))*['NaN']+ coments +(12-len(coments))*['NaN']
len(cols)

132